<a href="https://colab.research.google.com/github/MachuEngine/python-bigdata-colab-practice/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>#  HW1. 관측 이래 최고 기온 발생 날짜 및 값 구하기
학번: A70064, 이름: 안종민

In [ ]:
from google.colab import drive
import os
import csv

### 환경 설정 및 한글 폰트 로드

1. **라이브러리 임포트**
- `google.colab.drive`: Colab에서 내 Google Drive를 마운트해 파일에 접근하기 위해 사용
- `os`: 파일 경로를 다루거나 현재 작업 디렉토리를 확인할 때 사용
- `csv`: CSV 형식의 데이터를 읽고 쓸 때 사용

In [ ]:
drive.mount('./gdrive', force_remount=True)
print(os.getcwd())

Mounted at ./gdrive
/content


### Google Drive 마운트
- gdrive로 My Google Drive를 마운트 합니다.

In [ ]:
f = open('./gdrive/MyDrive/Colab Notebooks/ch4_temperature/temperature.csv', 'r', encoding='cp949')
data = csv.reader(f, delimiter=',')

# 헤더 건너 뛰기
# next(data) 를 호출하면 첫 번째 행(헤더)을 반환하면서 이터레이터 내부 포인터가 다음 행으로 넘어감.
header = next(data)  # ['날짜', '지점', '평균기온(℃)', '최저기온(℃)', '최고기온(℃)']

# 최고기온 찾기 및 오류 처리
max_temp = None
max_row  = None
errors   = []

# data 순회
for lineno, row in enumerate(data, start=2):
  # 빈 행은 건너뛰기
  if not row:
    continue

  # 컬럼 개수가 기대치(여기선 최소 5개)보다 작으면 건너뛰기
  if len(row) < 5:
    continue

  try:
    row[-1] = float(row[-1])
  except ValueError:
    errors.append((lineno, row))
    continue

  # 최고기온 갱신
  if (max_temp is None) or (row[-1] > max_temp):
    max_temp = row[-1]
    max_row  = row

# 데이터 수집 종료
f.close()

### 메인 동작

1. **헤더 건너뛰기**
- 첫 줄은 컬럼명만 있고 실제 데이터가 아니므로 `next(data)` 로 스킵

2. **enumerate 사용**
- `start=2`로 설정해, 데이터 첫 행이 원본 CSV의 2번째 줄 번호와 일치하도록 함

3. **빈 행·컬럼 부족 행 스킵**
- `if not row or len(row) < 5: continue` 로 안전하게 처리

4. **try/except로 오류 기록**
- `float()` 변환 실패 시 errors.append((lineno, row))

5. **최고기온 갱신**
- `max_temp`가 `None`이거나, 현재 `temp`가 더 크면 업데이트

6. **파일 핸들 정리**
- `f.close()` 로 파일을 닫아 리소스를 해제

In [ ]:
# 결과 출력
print(f'기상 관측 이래, 최고 기온이 가장 높았던 날은 {max_row[0].strip()}로, {max_temp} 도 입니다.')

# 결측치/오류 결과 출력
print(f'결측 및 오류가 발생한 일수는 {len(errors)}일입니다.')

기상 관측 이래, 최고 기온이 가장 높았던 날은 2018-08-01로, 39.6 도 입니다.
결측 및 오류가 발생한 일수는 759일입니다.


### 결과 출력

1. `.strip()` 사용하여 연도 출력 시 생기는 앞뒤 space를 제거

#  1. 에러 발생 상황 및 원인

#### 1) **헤더 포함 여부 미처리**  
   - `csv.reader` 로 읽은 `data`의 첫 행이 헤더(컬럼명)인데, 이를 건너뛰지 않으면 숫자 변환 시도 시 `ValueError` 발생  

#### 2) **빈 행(Empty row) 또는 컬럼 누락**  
   - CSV 파일에 공백 줄이 있거나, 쉼표 개수가 달라 한 줄이 `[]` 또는 `['…']` 형태로 읽히면  
   - `row[-1]` 접근 시 `IndexError: list index out of range` 에러 발생  

#### 3) **비숫자(Non-numeric) 문자열**  
   - 최고기온 컬럼에 `""`(빈 문자열) 혹은 `"-"` 같은 결측치 마커가 있을 경우  
   - `float(row[-1])` 변환 시 `ValueError: could not convert string to float` 발생  

<br>

---

<br>

# 2. 해결 방안

#### 1) **헤더 건너뛰기**  
   ```python
   header = next(reader)
   ```
- 첫 번째 행(컬럼명)을 꺼내어 변수에 저장하고, 데이터 수집을 위한 반복문에서는 실제 데이터만 처리하여 해결할 수 있습니다.

#### 2) **빈 행 및 컬럼 개수 검사**
  ```python
  if not row:
    continue
  if len(row) < 5:    # ['날짜','지점','평균','최저','최고'] 최소 5개
    continue
  ```
- row가 빈 리스트일 때나, 컬럼 개수가 예상보다 적을 때 해당 행 건너뜁니다.

#### 3) **ValueError 처리 (결측치 로그 및 건너뛰기)**
  ```python
    try:
        row[-1] = float(row[-1])
    except ValueError:
        errors.append((lineno, row))
        continue
  ```
- 필요한 데이터인 마지막 컬럼(최고기온)만 실수 변환 시도합니다.
- 실패할 경우 예외처리를 위하여 errors 리스트에 (행번호, 원본행) 형태로 기록 후 다음 행으로 건너뜁니다.